# Differential gene expression analysis - code example for single-cell specific

Using the data set Misharin.

## 1. Environment setup

In [1]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [2]:
%%R
library(MAST)

## 2. Preparing the data set

In [3]:
adata = sc.read("/home/sch/schonner/MaPra/misharin_Emma_for_merging.h5ad")
adata

AnnData object with n_obs × n_vars = 15365 × 27998
    obs: 'batch', 'author_annotation', 'manual_celltype_annotation', 'condition'

### We will need label (which contains the condition label), replicate and cell_type columns of the .obs

---> condition, batch and manual_celltype_annotation in this data set

---> relabel the columns for easier understanding

In [4]:
adata.obs = adata.obs.rename(columns={"condition": "label", "batch": "replicate", "manual_celltype_annotation": "cell_type"})
adata.obs[:5]

,replicate,author_annotation,cell_type,label
SC14_AAACCTGAGCGTTCCG,0,B cells,B-cells,control
SC14_AAACCTGAGGACATTA,0,B cells,B-cells,control
SC14_AAACCTGAGTCGTTTG,0,Alveolar macrophages,Alveolar macrophages,control
SC14_AAACCTGCACATCCGG,0,Classical monocytes,Classical monocytes,control
SC14_AAACCTGCACTAGTAC,0,T cells,T-cells,control


### We will need to work with raw counts so we check that .X indeed contains raw counts and put them into the counts layer of our AnnData object.

In [5]:
np.max(adata.X)

19899.0

In [6]:
adata.layers["counts"] = adata.X.copy()

### Rudimentary quality control (taken from the book chapter):

In [7]:
sc.pp.filter_cells(adata, min_genes=200)   # filter cells which have less than 200 genes
sc.pp.filter_genes(adata, min_cells=3)     # filter genes which were found in less than 3 cells
adata

AnnData object with n_obs × n_vars = 15365 × 17832
    obs: 'replicate', 'author_annotation', 'cell_type', 'label', 'n_genes'
    var: 'n_cells'
    layers: 'counts'

# 3. Single-cell specific

In [8]:
adata.X = adata.layers["counts"].copy()

In [9]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

#### Defining a helper function:

In [10]:
def prep_anndata(adata_):
    def fix_dtypes(adata_):
        df = pd.DataFrame(adata_.X.A, index=adata_.obs_names, columns=adata_.var_names)
        df = df.join(adata_.obs)
        return sc.AnnData(df[adata_.var_names], obs=df.drop(columns=adata_.var_names))

    adata_ = fix_dtypes(adata_)
    sc.pp.filter_genes(adata_, min_cells=3)
    return adata_

### 3.1. One group

In [11]:
adata_AT1 = adata[adata.obs["cell_type"] == "AT1"].copy()
adata_AT1

AnnData object with n_obs × n_vars = 66 × 17832
    obs: 'replicate', 'author_annotation', 'cell_type', 'label', 'n_genes'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [12]:
sc.pp.filter_genes(adata_AT1, min_cells=3)
adata_AT1

AnnData object with n_obs × n_vars = 66 × 8467
    obs: 'replicate', 'author_annotation', 'cell_type', 'label', 'n_genes'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [13]:
adata_AT1 = prep_anndata(adata_AT1)   # filter both objects
adata_AT1

AnnData object with n_obs × n_vars = 66 × 8467
    obs: 'replicate', 'author_annotation', 'cell_type', 'label', 'n_genes'
    var: 'n_cells'

#### Define a separate function that we use for the analysis:

In [14]:
adata_AT1.obs["cell_type"] = [
    ct.replace(" ", "_") for ct in adata_AT1.obs["cell_type"]
]
adata_AT1.obs["cell_type"] = [
    ct.replace("+", "") for ct in adata_AT1.obs["cell_type"]
]

In [15]:
%%R
find_de_MAST_RE <- function(adata_){
    # create a MAST object
    sca <- SceToSingleCellAssay(adata_, class = "SingleCellAssay")
    print("Dimensions before subsetting:")
    print(dim(sca))
    print("")
    # keep genes that are expressed in more than 10% of all cells
    sca <- sca[freq(sca)>0.1,]
    print("Dimensions after subsetting:")
    print(dim(sca))
    print("")
    # add a column to the data which contains scaled number of genes that are expressed in each cell
    cdr2 <- colSums(assay(sca)>0)
    colData(sca)$ngeneson <- scale(cdr2)
    # store the columns that we are interested in as factors
    label <- factor(colData(sca)$label)
    # set the reference level
    label <- relevel(label,"control")   # ---> needs to be consistent with the data set
    colData(sca)$label <- label
    celltype <- factor(colData(sca)$cell_type)
    colData(sca)$celltype <- celltype
    # same for donors (which we need to model random effects)
    replicate <- factor(colData(sca)$replicate)
    colData(sca)$replicate <- replicate
    # create a group per condition-celltype combination
    colData(sca)$group <- paste0(colData(adata_)$label, ".", colData(adata_)$cell_type)
    colData(sca)$group <- factor(colData(sca)$group)
    # define and fit the model
    zlmCond <- zlm(formula = ~ngeneson + group + (1 | replicate), 
                   sca=sca, 
                   method='glmer', 
                   ebayes=F, 
                   strictConvergence=F,
                   fitArgsD=list(nAGQ = 0)) # to speed up calculations
    
    # perform likelihood-ratio test for the condition that we are interested in    
    summaryCond <- summary(zlmCond, doLRT='groupstim.AT1')
    # get the table with log-fold changes and p-values
    summaryDt <- summaryCond$datatable
    result <- merge(summaryDt[contrast=='groupstim.AT1' & component=='H',.(primerid, `Pr(>Chisq)`)], # p-values
                     summaryDt[contrast=='groupstim.AT1' & component=='logFC', .(primerid, coef)],
                     by='primerid') # logFC coefficients
    # MAST uses natural logarithm so we convert the coefficients to log2 base to be comparable to edgeR
    result[,coef:=result[,coef]/log(2)]
    # do multiple testing correction
    result[,FDR:=p.adjust(`Pr(>Chisq)`, 'fdr')]
    result = result[result$FDR<0.01,, drop=F]

    result <- stats::na.omit(as.data.frame(result))
    return(result)
}

#### Run the pipeline for AT1:

In [16]:
%%time
%%R -i adata_AT1 -o res
res <-find_de_MAST_RE(adata_AT1)

[1] "Dimensions before subsetting:"
[1] 8467   66
[1] ""
[1] "Dimensions after subsetting:"
[1] 5678   66
[1] ""

Error in generateHypothesis(hypothesis, colnames(object@coefD)) : 
  Term(s) 'groupstim.AT1 ,' not found.
Terms available: (Intercept) , ngeneson , groupcontrol.AT1 ,


RInterpreterError: Failed to parse and evaluate line 'res <-find_de_MAST_RE(adata_AT1)\n'.
R error message: "Error in generateHypothesis(hypothesis, colnames(object@coefD)) : \n  Term(s) 'groupstim.AT1 ,' not found.\nTerms available: (Intercept) , ngeneson , groupcontrol.AT1 ,"

In [ ]:
res[:5]res["gene_symbol"] = res["primerid"]
res["cell_type"] = "AT1"
sc_toolbox.tools.de_res_to_anndata(
    adata,
    res,
    groupby="cell_type",
    score_col="coef",
    pval_col="Pr(>Chisq)",
    pval_adj_col="FDR",
    lfc_col="coef",
    key_added="MAST_AT1",
)

In [ ]:
adata_copy = adata.copy()

## 4. Visualization

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

### Define helper function for heatmaps:

In [ ]:
FDR = 0.01
LOG_FOLD_CHANGE = 1.5


def plot_heatmap(adata, group_key, group_name="cell_type", groupby="label"):
    cell_type = "_".join(group_key.split("_")[1:])
    res = sc.get.rank_genes_groups_df(adata, group=cell_type, key=group_key)
    res.index = res["names"].values
    res = res[
        (res["pvals_adj"] < FDR) & (abs(res["logfoldchanges"]) > LOG_FOLD_CHANGE)
    ].sort_values(by=["logfoldchanges"])
    print(f"Plotting {len(res)} genes...")
    markers = list(res.index)
    sc.pl.heatmap(
        adata[adata.obs[group_name] == cell_type].copy(),
        markers,
        groupby=groupby,
        swap_axes=True,
    )

### Plot heatmap for AT1:

In [ ]:
plot_heatmap(adata, "MAST_AT1")